# **Compile pre-processing outputs into one file**

By Bridget Bittmann

Date created: 04/04/2022

Date modified: 06/01/2022

In [1]:
## --------------- ##
## IMPORT PACKAGES ## 
## --------------- ##

import pandas as pd # to work with dataframe
import os # for file paths
import glob # read in a folder of csv
import numpy as np # basic statistics


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
## NAVIGATE TO YOUR DIRECTORY ##
%cd gdrive/MyDrive/spatial_colab/datasets/
%ls

/content/gdrive/MyDrive/spatial_colab/datasets
climate_stats/         irrigation_companies/  lcmap_files/   subset_test_shp/
diversion_timeseries/  irrig_lbrb/            masked/
extra_figures/         IrrMapper/             output_files/
hydromet_data/         LBRB_shp/              POUs/


In [4]:
## -------------------- ##
## IMOPORT ALL DATASETS ##
## -------------------- ##

div_files = sorted(glob.glob('diversion_timeseries/final_stats/*.csv'))
land_files = sorted(glob.glob('lcmap_files/final_metrics/*.csv'))
land_bpbc = sorted(glob.glob('lcmap_files/final_metrics/bpbc/*.csv'))
climate_files = sorted(glob.glob('climate_stats/final/*.csv'))
climate_bpbc = sorted(glob.glob('climate_stats/bpbc_final/*.csv'))
hydromet = pd.read_csv('hydromet_data/mode_input_hydromet.csv')
common_name_flow = pd.read_csv('diversion_timeseries/relates/name_dictionary_flow.csv')
common_name_spatial = pd.read_csv('diversion_timeseries/relates/name_dictionary_spatial.csv')
POUSize = pd.read_csv('diversion_timeseries/relates/POUSize.csv')
storage = pd.read_csv('diversion_timeseries/final_stats/accounting/accounting.csv')

In [5]:
## --------------------------------- ##
## CREATE COMMON NAME TO MERGE FILES ## 
## --------------------------------- ##

# Import discharge data 

div_data = []
for i in div_files:
  discharge = pd.read_csv(i)
  discharge = discharge.rename({'Name':'DivName'}, axis=1)
  discharge['DivName'] = discharge['DivName'].str.replace(r"\(.*\)", "")
  div_data.append(discharge)

bpbc = pd.read_csv('diversion_timeseries/bpbc/bpbc_totals.csv')
bpbc = bpbc.rename({'Acre-feet':'Acre_feet'},axis=1)
div = pd.concat(div_data)
# div_bpbc = pd.concat([div,bpbc])
# div_bpbc = div_bpbc.iloc[:,[1,2,3]]
# Dicharge data dictionary

div_dict = dict(zip(common_name_flow['DiversionName'], common_name_flow['NewName']))
div['Name'] = div['DivName'].map(div_dict)
div = div.drop(['Unnamed: 0', 'DivName'], axis=1)

# Import land use change data

land_data = []
ld_bpbc = []
for i in land_files:
  land_data.append(pd.read_csv(i))
for i in land_bpbc:
  ld_bpbc.append(pd.read_csv(i))

land_bpbc = pd.concat(ld_bpbc)
land = pd.concat(land_data)

# Land use dictionary

shape_dict = dict(zip(common_name_spatial['WaterRight'], common_name_spatial['NewName']))
land['Name'] = land['DivName'].map(shape_dict)
land = land.drop(['Unnamed: 0', 'DivName'], axis=1)
POUSize['Name'] = POUSize['WaterRight'].map(shape_dict)
POUSize = POUSize.drop(['WaterRight'], axis=1)
# Import climate zonal stats

clim_data = []
clim_bpbc = []
for i in climate_files:
  clim_data.append(pd.read_csv(i))
for i in climate_bpbc:
  clim_bpbc.append(pd.read_csv(i))
clim = pd.concat(clim_data)
climate_bpbc = pd.concat(clim_bpbc)

# Use shapefile dictionary on climate data

clim['Name'] = clim['DIV_NAME'].map(shape_dict)
clim = clim.dropna().drop(['Unnamed: 0', 'DIV_NAME'], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


In [6]:
## -------------------------------------- ## 
## MERGE DIVERSION DATA TO MATCH THE POUS ## 
## -------------------------------------- ## 


# Create function to do this

def merge_flows(data, name):
  '''
  This function will merge two different flow datasets into one for completely overlapping POUs.

  Variables:
  data : The full diversion dataset
  name : A string of the new name for each POU.
  '''
  old_df = data[data['Name']== name].reset_index().drop('index', axis=1)
  new_df = pd.DataFrame()
  new_df['Year'] = old_df['Year'].unique()
  new_df['Name'] = old_df['Name'][0:34]
  sums = old_df.groupby('Year').sum().reset_index()
  new_df['Diversion (cfs)'] = sums['Diversion (cfs)']
  new_df['Acre_feet'] = sums['Acre_feet']

  startday = []
  start_date = []
  endday = []
  range = []
  end_date = []

  for i in new_df['Year']:
    yearly = old_df[old_df['Year'] == i]
    start = np.min(yearly['StartDayofYear'].values)
    startdate = yearly['StartDate'][yearly['StartDayofYear']==start].values
    end = np.max(yearly['EndDayofYear'].values)
    enddate = yearly['EndDate'][yearly['EndDayofYear']==end].values
    startday.append(start)
    endday.append(end)
    range.append(end-start)
    start_date.append(startdate[0])
    end_date.append(enddate[0])

  new_df['StartDate'] = start_date
  new_df['StartDayofYear'] = startday
  new_df['EndDate'] = end_date
  new_df['EndDayofYear'] = endday
  new_df['Range'] = range

  return new_df

In [7]:
## ------------------------------------- ##
## MERGE DIVERSION DATASET WITH NEW DATA ##
## ------------------------------------- ##

# Create a list of names that have completely shared POUs
merge_names = ['Shipley and Wagner Pumps', 'Rossi Mill and Meeves Canals', 'Boise City Parks']

merged = []
for i in merge_names:
  new = merge_flows(div, i)
  div = div[div['Name'] != i] #Remove old dataframes from full dataset
  div = pd.concat([div, new])
  merged.append(new)

div = div.sort_values(by=['Name', 'Year']).reset_index().drop('index',axis=1)

In [ ]:
## BPBC Merging ##

relates = pd.read_csv('diversion_timeseries/bpbc/bpbc_relate.csv')

# Dicharge data dict
key_list = list(bpbc['DiversionNa'])
dict_lookup = dict(zip(relates['Discharge'], relates['NewName']))
bpbc['Name'] = [dict_lookup[item] for item in key_list]

# Land use change dict
key_list2 = list(land_bpbc['DivName'])
dict_lookup2 = dict(zip(relates['Shape'], relates['NewName']))
land_bpbc['Name'] = [dict_lookup2[item] for item in key_list2]

key_list3 = list(climate_bpbc['DIV_NAME'])
dict_lookup3 = dict(zip(relates['Shape'], relates['NewName']))
climate_bpbc['Name'] = [dict_lookup2[item] for item in key_list2]

## Flow data
bpbc = bpbc.drop('DiversionNa', axis=1)
div_bpbc = pd.concat([div, bpbc])
all_div = pd.DataFrame(div_bpbc[['Year', 'Name', 'Acre_feet']])
all_div = all_div.sort_values(by=['Name', 'Year'])

## Land use data
land_bpbc = land_bpbc.drop(['Unnamed: 0', 'DivName'], axis=1)
all_land = pd.concat([land_bpbc,land])

## Climate data
climate_bpbc = climate_bpbc.drop(['Unnamed: 0','DIV_NAME'], axis=1)
all_clim = pd.concat([climate_bpbc, clim])

In [ ]:
## ------------------------------- ## 
## MERGE THREE FILES INTO ONE FILE ##
## ------------------------------- ## 

land_div = div.merge(land, left_on=['Year', 'Name'], right_on=['dates','Name'], how='left')
full_df = land_div.merge(clim, left_on=['Year','Name'], right_on=['Year', 'Name'], how='left').sort_values(by=['Name', 'Year'])
full_df = full_df.merge(hydromet, left_on='Year', right_on='Year', how='left').drop(['Unnamed: 0', 'dates'], axis=1)
full_df = full_df.merge(POUSize, left_on = 'Name', right_on = 'Name', how = 'left')
## --------------------------------------- ##
## Export the full csv file for model in R ##
## --------------------------------------- ## 

# Full dataframe export
out_path = 'output_files/merged/model_input.csv'
full_df.to_csv(out_path)

# Individual dataframe export

names = full_df['Name'].unique()
for i in names:
  df = full_df[full_df['Name'] == i]
  out_path = os.path.join('output_files/'+i+'.csv')
  df.to_csv(out_path)

In [ ]:
## ------------------------------------ ## 
## MERGE THREE FILES INTO ONE FILE BPBC ##
## ------------------------------------ ## 

land_div = all_div.merge(all_land, left_on=['Year', 'Name'], right_on=['dates','Name'], how='left')
full_df = land_div.merge(all_clim, left_on=['Year','Name'], right_on=['Year', 'Name'], how='left').sort_values(by=['Name', 'Year'])
full_df = full_df.merge(hydromet, left_on='Year', right_on='Year', how='left').drop(['Unnamed: 0', 'dates'], axis=1)

# Get rid of New York data because using BPBC data
full_df = full_df[full_df['Name'] != 'New York Canal']
print(full_df['Name'].unique())
display(full_df)
## --------------------------------------- ##
## Export the full csv file for model in R ##
## --------------------------------------- ## 

# Full dataframe export
out_path = 'output_files/merged/bpbc_model_input.csv'
full_df.to_csv(out_path)

['Andrews' 'Atwell' 'Ballentyne Canal' 'Barber pumps' 'Baxter'
 'Boise City Canal' 'Boise City Parks' 'Boise Valley Canal'
 'Bowman and Swisher' 'Bubb Canal' 'Caldwell Highline Canal'
 'Caldwell Lowline Canal' 'Campbell Canal' 'Canyon County Canal'
 'Capitol View Canal' 'Conway-Hamming Canal' 'Crawforth' 'Division No2'
 'Division No3' 'Division No4' 'Division No5' 'Eagle Island State Park'
 'Ester Simplot' 'Eureka No1 Canal' 'Eureka No2 Canal' 'Fairview Acres'
 'Farmers Union Canal' 'Golden Gate Canal' 'Graham-Gilbert Canal' 'Haas'
 'Hart-Davis Canal' 'Island Highline Canal' 'Lemp Canal'
 'Little Pioneer Canal' 'Lower Center Point' 'Mace-Catlin Canal'
 'Mace-Mace Canal' 'Mammon' 'McConnel Island' 'McCurry Pump'
 'McManus and Teater Canal' 'Middleton Canal' 'Nampa and Meridian'
 'New Dry Creek Canal' 'New Union Canal' 'Parma Ditch'
 'Penitentiary Canal' 'Phyllis Canal' 'Quinns Pond' 'Ridenbaugh Canal'
 'River Run' 'Riverside Canal' 'Riverside Village'
 'Rossi Mill and Meeves Canals' 'Se

,Year,Name,Acre_feet,class1_urban,class2_crops,contagion,largest_patch_index,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et,LP_inflows,Max_Fill,Carryover
0,1988,Andrews,3208.271828,0.900766,97.462843,92.500272,96.952410,104.597045,68.048646,27.473194,32.672246,0.793502,532442.53,550214.71,177743
1,1989,Andrews,3554.388325,0.930791,97.342741,92.093882,96.832307,196.258921,108.219621,26.105711,31.477965,0.620083,825046.62,848992.87,140108
2,1990,Andrews,3642.652990,0.945804,97.327729,92.047765,96.817295,49.204056,136.184194,26.393551,31.651480,0.842326,621960.95,706666.06,290548
3,1991,Andrews,5992.079872,0.960817,97.357754,92.179285,96.847320,66.634154,142.997269,25.836531,31.404859,0.821943,575959.32,519360.80,284374
4,1992,Andrews,3869.760950,0.975829,97.297703,92.047057,96.802282,105.404923,90.977120,27.377155,31.580158,0.892358,416426.07,376107.62,86262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,2016,Warm Springs Canal,2020.269101,68.982088,29.139362,58.409110,67.365662,177.823397,118.912439,25.814988,32.056601,0.992555,899234.72,930232.27,330434
2042,2017,Warm Springs Canal,1897.710142,69.025775,29.095675,58.035792,67.496723,235.507123,216.908014,25.170508,32.851701,0.915755,1668656.90,925232.25,356816
2043,2018,Warm Springs Canal,1765.491656,68.239406,29.488860,62.428081,66.797728,133.191616,164.638589,25.413196,32.068256,0.930784,931793.23,935433.87,519091
2044,2019,Warm Springs Canal,1753.610637,74.967235,22.193097,65.277106,72.433377,210.864301,193.219122,24.230707,31.126554,0.915387,1054517.82,930449.84,371675
